In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

/content/drive/.shortcut-targets-by-id/1bqqm9-0FMOqiqyH_NIQBh-lJ36pWa7aX/Masters_Project/Datasets/OASIS3


In [4]:
!ls

 data		   oasis-scripts	'Usage Instructions.gdoc'
 downloaded_data   t1w_non_ad_jpgs
 fa20		   t1w_non_ad_patients


In [5]:
!pip install -r ../../Code/ganomaly/requirements.txt

In [6]:
!pip install mkl-fft

In [7]:
!pip install Pillow

# imports

In [8]:
import os
import glob
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


In [9]:
from PIL import Image
#import cv2
import os.path
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [10]:
print(torch.__version__)

1.2.0


In [11]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [77]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [104]:
train_image_number = 140 # Image slice from MRI scans to use for training data

base_folder = '/pyt_oasis3'
non_ad_jpg_folder = './t1w_non_ad_jpgs/' + str(train_image_number) + '/'
#ad_jpg_folder = './'

class Oasis3_train_normal(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3_train_normal, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set
        
        self.data = []
        self.targets = []
        
        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.data.append(resized)
                self.targets.extend(str(0)) # based on the jpg_folder in for loop
                break
        self.data = np.vstack(self.data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.data = self.data.transpose((0, 1, 2))  # convert to HWC
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

In [105]:
a=Oasis3_train_normal(base_folder, False, transform=transform)

./t1w_non_ad_jpgs/140/OAS30074_ses-d0049_run-01_T1w.jpg


In [106]:
# OpenCV ---
# Original: (176, 256, 3)
# Re-shaped up: (3, 256, 256)
# After transform: (3, 256, 256)
# PIL ---
# Shape : (256, 256)
a.data[0].shape

(256, 256)

# PIL image reading

In [96]:
data = []
with open ('./t1w_non_ad_jpgs/140/OAS30074_ses-d0049_run-01_T1w.jpg', 'rb') as f:
  img = Image.open(f)
  dsize = (256, 256)
  resized = img.resize(dsize)
  data.append(resized)

In [102]:
data[0].size

(256, 256)

# CIFAR data representation testing

In [55]:
from torchvision.datasets import CIFAR10

In [57]:
transform = transforms.Compose(
    [
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

dataset = {}
dataset['train'] = CIFAR10(root='./', train=True, download=True, transform=transform)



0it [00:00, ?it/s]

  0%|          | 0/170498071 [00:00<?, ?it/s]



  0%|          | 57344/170498071 [00:00<05:29, 516692.23it/s]

  0%|          | 286720/170498071 [00:00<04:14, 668180.87it/s]

  1%|          | 1171456/170498071 [00:00<03:03, 922090.63it/s]

  2%|▏         | 4022272/170498071 [00:00<02:08, 1299237.00it/s]

  5%|▌         | 9019392/170498071 [00:00<01:28, 1834651.66it/s]

  8%|▊         | 14475264/170498071 [00:00<01:00, 2580404.68it/s]

 12%|█▏        | 20045824/170498071 [00:00<00:41, 3608584.65it/s]

 15%|█▌        | 25583616/170498071 [00:01<00:28, 5003845.87it/s]

 18%|█▊        | 31039488/170498071 [00:01<00:20, 6855204.30it/s]

 21%|██▏       | 36577280/170498071 [00:01<00:14, 9258802.64it/s]

 25%|██▍       | 42213376/170498071 [00:01<00:10, 12285662.64it/s]

 28%|██▊       | 47898624/170498071 [00:01<00:07, 15961697.38it/s]

 32%|███▏      | 53747712/170498071 [00:01<00:05, 20414458.50it/s]

 35%|███▍      | 58990592/170498071 [00:01<00:04, 24991985.35it/s]

 38%|███▊      | 64495616/170498071 [00:01<00:03, 29816226.17it/s]


Extracting ./cifar-10-python.tar.gz to ./


In [59]:
type(dataset['train'])

torchvision.datasets.cifar.CIFAR10

In [65]:
elt = dataset['train'][0]

In [71]:
type(elt)

tuple

In [76]:
elt[0].shape

torch.Size([3, 32, 32])